# Closed loop laser scan with NI-DAQmx

## Setup

In [ ]:
from pylabnet.utils.logging.logger import LogClient
from pylabnet.network.client_server import high_finesse_ws7
from pylabnet.network.client_server import nidaqmx_card
from pylabnet.utils.sweeper import Sweep1D

Enter host and port parameters here:

In [ ]:
logger_host = '0.0.0.0'
logger_port = 0
wavemeter_host = '0.0.0.0'
wavemeter_port = 0
nidaqmx_host = '0.0.0.0'
nidaqmx_port = 0

Instantiate all clients

In [ ]:
log = LogClient(
    host=logger_host,
    port=logger_port
)
wm = high_finesse_ws7.Client(
    host=wavemeter_host,
    port=wavemeter_port,
)
ni = nidaqmx_card.Client(
    host=nidaqmx_host,
    port=nidaqmx_port
)
exp = Sweep1D(logger=log)

## Experiment

In [ ]:
f0 = 406.7
f1 = 406.71
pts = 101
exp.set_parameters(min=f0, max=f1, pts=pts)

In [ ]:
exp.run()